<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 4: Web Scraping Job Postings

Detailed instructions are in Readme.md 
My answers for this Project are split into several notebooks to make it easier to review key sections and make changes where needed. In 1.0, I start with the scraping of the jobs posting data.

In [1]:
%load_ext blackcellmagic

In [2]:
import numpy as np
import os
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib

from bs4 import BeautifulSoup
from time import sleep

## 1. SCRAPPING FOR JOBS DATA

### 1.1 Scrapping for links matching search terms: 
Terms used: data scientist, data science, data analyst, data analytics, business analytics, financial analytics, marketing analytics, data visualisation, data operations, data strategist, data engineer, data architect, data manager, data intern, data lead, data consultant, machine learning, deep learning, big data, business intelligence (To try to get at least 1,000 unique links).
Source: https://www.mycareersfuture.sg

In [3]:
links = [
    "https://www.mycareersfuture.sg/search?search=data%20scientist&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20science&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20analyst&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20analytics&page={}",
    "https://www.mycareersfuture.sg/search?search=business%20analytics&page={}",
    "https://www.mycareersfuture.sg/search?search=financial%20analytics&page={}",
    "https://www.mycareersfuture.sg/search?search=marketing%20analytics&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20visualisation&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20operations&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20strategist&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20engineer&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20architect&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20manager&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20lead&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20consultant&page={}",
    "https://www.mycareersfuture.sg/search?search=data%20intern&page={}",
    "https://www.mycareersfuture.sg/search?search=machine%20learning&page={}",
    "https://www.mycareersfuture.sg/search?search=deep%20learning&page={}",
    "https://www.mycareersfuture.sg/search?search=big%20data&page={}",
    "https://www.mycareersfuture.sg/search?search=business%20intelligence&page={}",
]

job_postings = []

for link in links:
    for pages in range(0, 30 + 1):
        driver = webdriver.Chrome(executable_path="../data/chromedriver/chromedriver")

        driver.get(link.format(pages))

        sleep(5)

        html = BeautifulSoup(driver.page_source, "lxml")

        for job_card in html.find_all(
            "div", id=lambda x: x and x.startswith("job-card-")
        ):
            job_postings.append(
                "https://www.mycareersfuture.sg" + job_card.find("a")["href"]
            )

        driver.close()

In [4]:
#Removing duplicate links
job_postings = list(dict.fromkeys(job_postings))

In [5]:
len(job_postings)
# Close to requirement for at least 1,000 job listings. Moving to next stage

975

### 1.2 CREATING CSV FILE WITH JOB LISTINGS

In [40]:
company = []
title = []
salary = []
responsibilities = []
requirements = []
seniority = []
employment_type = []
category = []
address = []
failed_url = []

driver = webdriver.Chrome(executable_path="../data/chromedriver/chromedriver")

for index, link in enumerate(job_postings):

    try:
        driver.get(link)
        sleep(3)
        job_page = driver.page_source

        soup = BeautifulSoup(job_page, "lxml")

        company.append(soup.find("p", {"name": "company"}))
        title.append(soup.find("h1", {"id": "job_title"}))
        salary.append(soup.find("div", {"class": "lh-solid"}))
        responsibilities.append(soup.find("div", {"id": "job_description"}))
        requirements.append(soup.find("div", {"id": "requirements"}))
        seniority.append(soup.find("p", {"id": "seniority"}))
        employment_type.append(soup.find("p", {"id": "employment_type"}))
        category.append(soup.find("p", {"id": "job-categories"}))
        address.append(soup.find("p", {"id": "address"}))

    except:
        failed_url.append(link)

In [41]:
raw = [company, title, salary, responsibilities, requirements, seniority, employment_type, category, address]

In [42]:
clean = {}

for index, items in enumerate(raw):
    clean[index] = []
    for elements in items:
        try:
            clean[index].append(elements.text)
        except:
            clean[index].append(None)

In [43]:
data_jobs = pd.DataFrame(clean)

In [45]:
data_jobs.columns = [
    "Company",
    "Title",
    "Salary_Range",
    "Responsibilities",
    "Requirements",
    "Seniority",
    "Employment_Type",
    "Category",
    "Address",
]

In [46]:
data_jobs.head()

,Company,Title,Salary_Range,Responsibilities,Requirements,Seniority,Employment_Type,Category,Address
0,HITACHI CONSULTING SINGAPORE PTE. LTD.,Senior Consultant (Data Scientist),"$6,000to$9,000",Roles & Responsibilities Problem solver with c...,Requirements Sound knowledge of machine learni...,Senior Executive,"Permanent, Contract",Information Technology,"PLAZA 8 @ CBP, 1 CHANGI BUSINESS PARK CRESCENT..."
1,TRAVELOKA SERVICES PTE. LTD.,Data Science Lead,"$10,000to$15,000","Roles & ResponsibilitiesAs a Data scientist, y...",RequirementsWe are looking for someone with: ...,Manager,Full Time,Information Technology,None
2,TRAVELOKA SERVICES PTE. LTD.,Senior Level Data Scientist,"$8,300to$15,000",Roles & ResponsibilitiesJob Description As a D...,Requirements Solid programming skills and und...,Executive,Permanent,Information Technology,None
3,TRAVELOKA SERVICES PTE. LTD.,Mid - Senior Level Data Scientist,"$6,100to$10,700","Roles & ResponsibilitiesAs a Data scientist, y...",Requirements Solid programming skills and und...,Executive,Permanent,Information Technology,None
4,TRAVELOKA SERVICES PTE. LTD.,Mid Level Data Scientist,"$4,300to$7,600","Roles & ResponsibilitiesAs a Data scientist, y...",Requirements Solid programming skills and und...,Executive,Permanent,Information Technology,None


In [68]:
#data_jobs = data_jobs.to_csv('../data/jobs.csv')

### DATA CLEANING AND EDA TO BE CONTINUED IN 2.0-cch-project4-EDA